In [ ]:
import numpy as np
import os
import pandas as pd
import yaml
np.random.seed(42)
import os
from resum.polynomial_chaos_expansion import PCEMultiFidelityModel

In [ ]:
with open("./binary-black-hole/settings.yaml", "r") as f:
    config_file = yaml.safe_load(f)

version       = config_file["path_settings"]["version"]
path_out_cnp  = config_file["path_settings"]["path_out_cnp"]
path_out_pce = config_file["path_settings"]["path_out_pce"]
file_in=f'{path_out_cnp}/cnp_{version}_output.csv'
print(version, file_in)


In [ ]:

if not os.path.exists(path_out_pce):
   os.makedirs(path_out_pce)

# Set parameter name/x_labels -> needs to be consistent with data input file
x_labels        = config_file["simulation_settings"]["theta_headers"]
y_label_cnp     = 'y_cnp'
y_err_label_cnp = 'y_cnp_err'
y_label_sim     = 'y_raw'

# Set parameter boundaries
xmin = config_file["simulation_settings"]["theta_min"]
xmax = config_file["simulation_settings"]["theta_max"]
x_fixed = config_file["simulation_settings"]["theta_fixed"]
parameters={}
for i,x in enumerate(x_labels):
   parameters[x]=[xmin[i],xmax[i]]


In [ ]:
data=pd.read_csv(file_in)

In [ ]:

LF_cnp_noise=np.mean(data.loc[(data['fidelity']==0.) & (data['iteration']==0)][y_err_label_cnp].to_numpy())
HF_cnp_noise=np.mean(data.loc[(data['fidelity']==1.) & (data['iteration']==0)][y_err_label_cnp].to_numpy())

x_train_hf = data.loc[(data['fidelity']==1.) & (data['iteration']==0)][x_labels].to_numpy()
y_train_hf = data.loc[(data['fidelity']==1.) & (data['iteration']==0)][y_label_sim].to_numpy()

x_train_mf = data.loc[(data['fidelity']==1.) & (data['iteration']==0)][x_labels].to_numpy()
y_train_mf = data.loc[(data['fidelity']==1.) & (data['iteration']==0)][ y_label_cnp].to_numpy()

x_train_lf_sim = data.loc[(data['fidelity']==0.) & (data['iteration']==0)][x_labels].to_numpy()
y_train_lf_sim = data.loc[(data['fidelity']==0.) & (data['iteration']==0)][ y_label_sim].to_numpy()

x_train_lf = data.loc[(data['fidelity']==0.) & (data['iteration']==0)][x_labels].to_numpy()
y_train_lf = data.loc[(data['fidelity']==0.) & (data['iteration']==0)][ y_label_cnp].to_numpy()

In [ ]:
# Initialize the model
trainings_data = {"lf": [x_train_lf,y_train_lf], "mf": [x_train_mf, y_train_mf], "hf": [x_train_hf,y_train_hf]}

# Prior configurations
priors = {
    "lf": {"sigma_coeffs": 0.5, "sigma": 0.01},
    "mf": {"sigma_rho": 0.08, "sigma_coeffs_delta": 0.005, "sigma": 0.005},
    "hf": {"sigma_rho": 0.08, "sigma_coeffs_delta": 0.005, "sigma": 0.005}
}
print(trainings_data["hf"])


In [ ]:
# Initialize the multi-fidelity model
multi_fidelity_model = PCEMultiFidelityModel(trainings_data, priors, degree=1)

In [ ]:
# Build the model
multi_fidelity_model.build_model()
trace=multi_fidelity_model.run_inference()

In [ ]:
multi_fidelity_model.plot_trace()

In [ ]:
#data_test=pd.read_csv("{resum_path}/baseline_models_comparison/in/hf_validation_data_v1.2.csv")
#row_hf_test=data_test.index[data_test['fidelity'] == 1]
#row_lf_test=data_test.index[data_test['fidelity'] == 0]
#x_test = data_test.loc[row_hf_test][x_labels].to_numpy()
#y_test = data_test.loc[row_hf_test][y_label_sim].to_numpy()
#test_data = {"hf": [x_test,y_test]}

In [ ]:
#multi_fidelity_model.evaluate_mse(x_test, y_test)

In [ ]:
#multi_fidelity_model.plot_validation(x_test,y_test)

In [ ]:

#multi_fidelity_model.validate_coverage(x_test,y_test)